In [8]:
import json
import pandas as pd
import numpy as np
import sys
import os
import shutil
sys.path.insert(0,'/Users/sanjivjha/IMP-OIC/')
from utils.extractframes import FrameExtractor
import graphene
from inference.gpt_ask import run_gpt
directory_path = 'STAR_eval/Charades'


KeyError: 'open_ai_api_key'

In [23]:
QA_ds = pd.read_json('STAR_eval/STAR_val.json')
QA_ds.head()

,question_id,question,video_id,start,end,answer,question_program,choices,situations
0,Interaction_T1_13,Which object was tidied up by the person?,6H78U,11.1,19.6,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The closet/cabine...","{'000206': {'rel_pairs': [['o000', 'o027'], ['..."
1,Interaction_T1_14,Which object was tidied up by the person?,6H78U,15.6,22.7,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The blanket.', 'c...","{'000289': {'rel_pairs': [['o000', 'o027'], ['..."
2,Interaction_T1_31,Which object was thrown by the person?,RNLTR,7.3,13.5,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The pillow.', 'ch...","{'000132': {'rel_pairs': [['o000', 'o019'], ['..."
3,Interaction_T1_32,Which object was put down by the person?,RNLTR,11.6,16.4,The shoe.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The food.', 'choi...","{'000202': {'rel_pairs': [['o000', 'o019'], ['..."
4,Interaction_T1_40,Which object was tidied up by the person?,VNQTH,2.7,9.2,The closet/cabinet.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The broom.', 'cho...","{'000110': {'rel_pairs': [['o000', 'o014'], ['..."


In [56]:
QA_ds_questions = QA_ds.loc[:, ['question_id','question','video_id','start','end','answer', 'choices']]

In [68]:
QA = QA_ds_questions.groupby(['question_id','video_id']).sum().reset_index()#.to_csv('STAR_eval/question_set.csv')

QA.head()

,question_id,video_id,question,start,end,answer,choices
0,Feasibility_T2_10,K7VCU,What else is the person able to do with the dish?,0.0,9.1,Take the dish.,"[{'choice_id': 0, 'choice': 'Put down the dish..."
1,Feasibility_T2_1027,SMVKB,What else is the person able to do with the door?,16.5,23.6,Close the door.,"[{'choice_id': 0, 'choice': 'Close the door.',..."
2,Feasibility_T2_1034,G4E4S,What else is the person able to do with the cl...,4.7,9.6,Wash the clothes.,"[{'choice_id': 0, 'choice': 'Sit on the clothe..."
3,Feasibility_T2_1046,MQGVS,What else is the person able to do with the bl...,3.0,9.7,Put down the blanket.,"[{'choice_id': 0, 'choice': 'Close the blanket..."
4,Feasibility_T2_1048,MQGVS,What else is the person able to do with the bl...,5.9,13.0,Put down the blanket.,"[{'choice_id': 0, 'choice': 'Put down the blan..."


In [74]:
def run_oic(question, choices):
    TEMP_DIR = "temp"
    CAM_PATH = "cam"
    OUT_DIR = "out"

    if os.path.isdir(TEMP_DIR):
        shutil.rmtree(TEMP_DIR) 

    for i in QA['video_id'].unique().values:
        print('processing {}'.format(i[0]))
        
        for video in os.scandir(directory_path):
            if video.is_file():
                if os.path.splitext(video.name)[0] == i[0]:
                    ex = FrameExtractor(30, video.path)
                    #ex.main()
                    g = graphene.Graphene(0.3, 0.6)

                    if not os.path.isdir(OUT_DIR):
                        os.mkdir(OUT_DIR)
                    text = os.path.splitext(video.name)[0]+'graph2text.txt'
                    img_path = directory_path+'/'+os.path.splitext(video.name)[0]+'-opencv'
                    visual = os.path.splitext(video.name)[0]+'tg.png'
                    #g.run_online(text)

                    g.classify_images(img_path)
                    graph_path = TEMP_DIR
                    g.generate_temporal_graph_frames(graph_path, img_path)

                    #g.generate_temporal_graph(graph_path)

                    g.tg.to_plot(os.path.join(OUT_DIR, visual))

                    g.tg.to_text(os.path.join(OUT_DIR,text))
                    
                    if os.path.isfile(OUT_DIR+'/'+text):
                        with open(OUT_DIR+'/'+text) as f:
                            context = f.readline()
                        
                            run_gpt(context, question + choices)

In [3]:
q_ids = QA['question_id'].unique()

for q in q_ids:
    choices = 'Options: '
    que = QA.query("question_id == '"+q+"'")
    video_id = que['video_id']
    question = que['question']
    start = que['start']
    end = que['end']
    for choice in que['choices'][0]:
        choices+=choice['choice']
    print(choices)

NameError: name 'QA' is not defined